# Triage False Positive Findings

This notebook demonstrates how to identify and mark false positive findings, specifically focusing on noseyparker_secret findings with generic password detections.

## Setup

First, let's import the required libraries and set up our connection to Hasura.

In [ ]:
import json
import os
import re

from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

# Configuration - Set your username here
username = "analyst"  # Change this to your username

# Set up the GraphQL client
hasura_url = os.getenv('HASURA_GRAPHQL_URL', 'http://hasura:8080/v1/graphql')
admin_secret = os.getenv('HASURA_ADMIN_SECRET', '')

transport = RequestsHTTPTransport(
    url=hasura_url,
    headers={'x-hasura-admin-secret': admin_secret}
)

client = Client(transport=transport, fetch_schema_from_transport=True)
print(f"Connected to Hasura at: {hasura_url}")
print(f"Username set to: {username}")

## Query noseyparker_secret Findings

Let's first pull all findings with finding_name = "noseyparker_secret" and examine their structure.

In [ ]:
# Query for noseyparker_secret findings
query = gql("""
    query {
        findings(where: {finding_name: {_eq: "noseyparker_secret"}}) {
            finding_id
            finding_name
            category
            severity
            object_id
            origin_type
            origin_name
            raw_data
            data
            created_at
            triage_id
        }
    }
""")

result = client.execute(query)
noseyparker_findings = result['findings']

print(f"Found {len(noseyparker_findings)} noseyparker_secret findings")

# Display first finding as example
if noseyparker_findings:
    print("\n=== Example Finding ===")
    example_finding = noseyparker_findings[0]
    for key, value in example_finding.items():
        if key in ['raw_data', 'data']:
            print(f"{key}: {json.dumps(value, indent=2)}")
        else:
            print(f"{key}: {value}")
else:
    print("No noseyparker_secret findings found")

## Filter for Generic Password Rules

Now let's filter findings where the raw_data.match.rule_name equals "Generic Password".

In [ ]:
# Filter findings with Generic Password rule
generic_password_findings = []

for finding in noseyparker_findings:
    raw_data = finding.get('raw_data', {})
    match_data = raw_data.get('match', {})
    rule_name = match_data.get('rule_name', '')

    if rule_name == "Generic Password":
        generic_password_findings.append(finding)

print(f"Found {len(generic_password_findings)} findings with 'Generic Password' rule")

# Show some examples
for i, finding in enumerate(generic_password_findings[:3]):
    print(f"\n=== Generic Password Finding {i+1} ===")
    print(f"Finding ID: {finding['finding_id']}")
    print(f"Rule Name: {finding['raw_data']['match']['rule_name']}")
    print(f"Matched Content: {finding['raw_data']['match'].get('matched_content', 'N/A')}")
    print(f"Snippet: {finding['raw_data']['match'].get('snippet', 'N/A')[:100]}...")

## Mark Generic Password Findings as False Positive

Let's mark all Generic Password findings as false positives by inserting records into the findings_triage_history table.

In [ ]:
# Mark Generic Password findings as false positive
if generic_password_findings:
    # Prepare triage history entries
    triage_entries = []
    for finding in generic_password_findings:
        triage_entries.append({
            "finding_id": finding['finding_id'],
            "username": username,
            "automated": True,
            "value": "false_positive"
        })

    # Insert triage entries
    mutation = gql("""
        mutation InsertTriageEntries($objects: [findings_triage_history_insert_input!]!) {
            insert_findings_triage_history(objects: $objects) {
                returning {
                    id
                    finding_id
                    username
                    value
                    timestamp
                }
            }
        }
    """)

    result = client.execute(mutation, variable_values={"objects": triage_entries})
    inserted_entries = result['insert_findings_triage_history']['returning']

    print(f"Successfully marked {len(inserted_entries)} Generic Password findings as false positive")

    # Show some examples
    for entry in inserted_entries[:3]:
        print(f"  - Finding ID {entry['finding_id']}: {entry['value']} by {entry['username']} at {entry['timestamp']}")
else:
    print("No Generic Password findings to mark as false positive")

## Filter for 'password123' Pattern

Now let's find findings where the matched_content matches a pattern for 'password123' and mark them as false positives.

In [ ]:
# Filter findings with 'password123' pattern
password123_pattern = re.compile(r'password123', re.IGNORECASE)
password123_findings = []

for finding in noseyparker_findings:
    raw_data = finding.get('raw_data', {})
    match_data = raw_data.get('match', {})
    matched_content = match_data.get('matched_content', '')

    if password123_pattern.search(matched_content):
        password123_findings.append(finding)

print(f"Found {len(password123_findings)} findings with 'password123' pattern")

# Show examples
for i, finding in enumerate(password123_findings[:3]):
    print(f"\n=== Password123 Finding {i+1} ===")
    print(f"Finding ID: {finding['finding_id']}")
    print(f"Rule Name: {finding['raw_data']['match'].get('rule_name', 'N/A')}")
    print(f"Matched Content: {finding['raw_data']['match'].get('matched_content', 'N/A')}")

## Mark password123 Findings as False Positive

Let's mark all findings containing 'password123' as false positives.

In [ ]:
# Mark password123 findings as false positive
if password123_findings:
    # Prepare triage history entries
    triage_entries = []
    for finding in password123_findings:
        triage_entries.append({
            "finding_id": finding['finding_id'],
            "username": username,
            "automated": True,
            "value": "false_positive"
        })

    # Insert triage entries
    mutation = gql("""
        mutation InsertTriageEntries($objects: [findings_triage_history_insert_input!]!) {
            insert_findings_triage_history(objects: $objects) {
                returning {
                    id
                    finding_id
                    username
                    value
                    timestamp
                }
            }
        }
    """)

    result = client.execute(mutation, variable_values={"objects": triage_entries})
    inserted_entries = result['insert_findings_triage_history']['returning']

    print(f"Successfully marked {len(inserted_entries)} password123 findings as false positive")

    # Show some examples
    for entry in inserted_entries[:3]:
        print(f"  - Finding ID {entry['finding_id']}: {entry['value']} by {entry['username']} at {entry['timestamp']}")
else:
    print("No password123 findings to mark as false positive")